In [ ]:
import tkinter as tk
from tkinter import ttk
import sqlite3

# Initialize database
def init_db():
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()

    # Create table with the updated structure if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id INTEGER PRIMARY KEY,
        task TEXT NOT NULL,
        category TEXT DEFAULT 'Other',
        completed BOOLEAN DEFAULT 0
    )
    """)

    # Check if the 'category' column exists
    cursor.execute("PRAGMA table_info(tasks)")
    columns = [info[1] for info in cursor.fetchall()]
    if 'category' not in columns:
        # Add the 'category' column if it doesn't exist
        cursor.execute("ALTER TABLE tasks ADD COLUMN category TEXT DEFAULT 'Other'")
        cursor.execute("ALTER TABLE tasks ADD COLUMN completed BOOLEAN DEFAULT 0")
    
    conn.commit()
    conn.close()

init_db()

# Add task function
def add_task():
    task = task_entry.get()
    category = category_combobox.get()
    if task:
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO tasks (task, category) VALUES (?, ?)", (task, category))
        conn.commit()
        conn.close()
        task_entry.delete(0, tk.END)
        category_combobox.set("Other")
        load_tasks()

# Delete task function
def delete_task():
    selected_task = task_list.curselection()
    if selected_task:
        task_id = task_list.get(selected_task[0]).split(".")[0]
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM tasks WHERE id=?", (task_id,))
        conn.commit()
        conn.close()
        load_tasks()

# Load tasks into the Listbox
def load_tasks():
    task_list.delete(0, tk.END)
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, task, category FROM tasks")
    tasks = cursor.fetchall()
    conn.close()

    for task in tasks:
        task_id, task_name, category = task
        task_list.insert(tk.END, f"{task_id}. {task_name} [{category}]")

# Initialize the app window
root = tk.Tk()
root.title("To-Do List App")
root.geometry("400x500")

# Widgets
task_entry = tk.Entry(root, width=30)
task_entry.pack(pady=10)

# Task category combobox
categories = ["Work", "Personal", "Urgent", "Other"]
category_combobox = ttk.Combobox(root, values=categories, state="readonly")
category_combobox.set("Other")
category_combobox.pack(pady=5)

add_button = tk.Button(root, text="Add Task", command=add_task)
add_button.pack(pady=5)

delete_button = tk.Button(root, text="Delete Task", command=delete_task)
delete_button.pack(pady=5)

task_list = tk.Listbox(root, width=50, height=15)
task_list.pack(pady=10)

# Load tasks on startup
load_tasks()

# Run the app
root.mainloop()
